In [18]:
from pyspark.sql import SparkSession
from delta import configure_spark_with_delta_pip

builder = SparkSession.builder \
    .appName("Delta Spark 4.0.0") \
    .master("local[*]") \
    .enableHiveSupport() \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

spark = configure_spark_with_delta_pip(builder).getOrCreate()

In [21]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import current_timestamp, input_file_name

df_customer = spark.read.csv("../data/Customer.csv", header = True, inferSchema=True)\
    .withColumn("fecha_carga", current_timestamp())\
    .withColumn("archivo_origen", input_file_name())
df_product = spark.read.csv("../data/Product.csv", header = True, inferSchema=True)\
    .withColumn("fecha_carga", current_timestamp())\
    .withColumn("archivo_origen", input_file_name())
df_so_detail = spark.read.csv("../data/SalesOrderDetail.csv", header = True, inferSchema=True)\
    .withColumn("fecha_carga", current_timestamp())\
    .withColumn("archivo_origen", input_file_name())
df_so_header = spark.read.csv("../data/SalesOrderHeader.csv", header = True, inferSchema=True)\
    .withColumn("fecha_carga", current_timestamp())\
    .withColumn("archivo_origen", input_file_name())
df_s_territory = spark.read.csv("../data/SalesTerritory.csv", header = True, inferSchema=True)\
    .withColumn("fecha_carga", current_timestamp())\
    .withColumn("archivo_origen", input_file_name())

In [ ]:
df_customer.show()
df_product.show()
df_so_detail.show()
df_so_header.show()
df_s_territory.show()

In [23]:
spark.sql("""
CREATE DATABASE IF NOT EXISTS dw_bronze
LOCATION 'D:/Curso - Python/Proyecto 2/Adventure Works/sql/DW_BRONZE'
""")

spark.sql("""
CREATE DATABASE IF NOT EXISTS dw_silver
LOCATION 'D:/Curso - Python/Proyecto 2/Adventure Works/sql/DW_SILVER'
""")

spark.sql("""
CREATE DATABASE IF NOT EXISTS dw_gold
LOCATION 'D:/Curso - Python/Proyecto 2/Adventure Works/sql/DW_GOLD'
""")

spark.sql("SHOW DATABASES").show()

+---------+
|namespace|
+---------+
|  default|
|dw_bronze|
|  dw_gold|
|dw_silver|
+---------+



In [ ]:
df_customer.write.format("delta").mode("overwrite").save("../sql/dw_bronze/customer")
df_product.write.format("delta").mode("overwrite").save("../sql/dw_bronze/product")
df_so_detail.write.format("delta").mode("overwrite").save("../sql/dw_bronze/so_detail")
df_so_header.write.format("delta").mode("overwrite").save("../sql/dw_bronze/so_header.write")
df_s_territory.write.format("delta").mode("overwrite").save("../sql/dw_bronze/s_terrotory.write")